## 좋아요 개수 기반 가중치 부여 

In [ ]:
# 좋아요 개수를 로그 변환
train_data['log_like_cnt'] = np.log(train_data['like_cnt'] + 1)

# 좋아요(로그) 개수의 분포 확인
train_data['log_like_cnt'].hist(bins=100)
train_data['log_like_cnt'].describe()

In [ ]:
# 좋아요 개수에 따라 구간 나누기
bins = [0, 1.098612, 2.079442, 3.218876, 11]  # 좋아요 개수 구간 설정
labels = [0, 0.1, 0.2, 0.3]  # 가중치 값 결정

# 'likes_range' 열 추가
train_data["like_weight"] = pd.cut(train_data["log_like_cnt"], bins=bins, labels=labels)  # 가중치가 설정된 열 추가
train_data

## 가중치 계산을 반영한 새로운 추천 함수

In [ ]:
def song_recommend_weighted_songs(tags, songs, tag_df, song_df, sim):
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim, get_cv(song_df), songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    #print(simi_songs)
    
    '''

    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1
    '''
       
    ts = tags
    
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    tag_simi_mean = []
    
    for simi_tags in simi_songs['tags']:
        songs_similality = []
        for input_tag in ts:
            # 유사도 + 가중치의 합을 구함
            songs_similality.append(get_tag_simi(input_tag, simi_tags, True) + train_data.loc[input_tag, 'like_weight'])
        tag_simi_mean.append(sum(songs_similality) / len(songs_similality))
    
    tag_simi_mean = pd.DataFrame(columns = ['tag_simi'], data = tag_simi_mean)
    sorted_idx = tag_simi_mean['tag_simi'].sort_values(ascending = False).index
    #for i in sorted_idx:
    #    print("원본: {} / simi_mean: {}".format(simi_songs.loc[i, 'song_id'], tag_simi_mean.loc[i, 'tag_simi']))
    tag_simi_mean = simi_songs.loc[sorted_idx]
    
    '''
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    
    for rec in simi_songs['song_id']:
        title_song = tag_songs[tag_songs['song_id'] == rec]
        title_index = title_song.index
        recommended.append(title_index[0])
    
    '''
    
    return tag_simi_mean.iloc[:10]